# Managing users in Amazon Rekognition Face collections

Amazon Rekognition can store information about detected faces in server-side containers known as collections. 
You can store individual faces and associate multiple individual faces with a single user. 

Individual faces are stored as face vectors, a mathematical representation of the face (not the actual image of the face).
Multiple face vectors can then be aggregated to create and store user vectors. 

User vectors are more robust representations, as they contain multiple face vectors with varying degrees of lighting, sharpness, poses, appearance differences, etc. 

Face matching with user vectors can improve accuracy by up to 45% compared to individual face vectors. You can use faces detected in images, stored videos, and streaming videos to search against stored face vectors and/or user vectors for face matching purposes.

### Environment Setup

First step let's import the necessary libraries to run the notebook and create an Amazon Rekognition client with Boto3.

In [1]:
import json, boto3, os
client = boto3.client("rekognition")

### Create a new collection

Before we begin to create users in Rekognition, we must have an existing face collection. 

In [2]:
collection_id='My_Face_Collection1' # Remember you must use a unique name if you are creating a new collection

In [3]:
def create_collection(collection_id):
    #Create a collection
    print('Creating collection:' + collection_id)
    response=client.create_collection(CollectionId=collection_id)
    print('Collection ARN: ' + response['CollectionArn'])
    print('Status code: ' + str(response['StatusCode']))
    print('Done...')
    
create_collection(collection_id)

Creating collection:My_Face_Collection1
Collection ARN: aws:rekognition:us-east-1:715319310744:collection/My_Face_Collection1
Status code: 200
Done...


### Confirm your collection creation

Let's display the collections in our account to verify the previous collection was completed correctly.

In [4]:
def list_collections():

    max_results=10

    print('Displaying collections...')
    response=client.list_collections(MaxResults=max_results)
    collection_count=0
    done=False

    while done==False:
        collections=response['CollectionIds']

        for collection in collections:
            print ("- "+ collection)
            collection_count+=1
        if 'NextToken' in response:
            nextToken=response['NextToken']
            response=client.list_collections(NextToken=nextToken,MaxResults=max_results)

        else:
            done=True
            
    return collection_count

collection_count=list_collections()

print("Collections: " + str(collection_count))

Displaying collections...
- My_Face_Collection1
- Riv-Prod-0
- collection_v4_1
- collection_v4_2
- collection_v4_3
- collection_v4_4
- collection_v4_5
- collection_v5
- collection_v5_1
- collection_v5_2
- collection_v5_3
- collection_v5_4
- collection_v5_5
- collection_v6_1
- collection_v6_2
- collection_v6_3
- user-vector-blog-collection
Collections: 17


### Create a new user

Once the collection is created, we can proceed to create a user. We are going to use the **create_user** method, which creates a new user in a collection and returns a unique user ID.

In [5]:
user_id = "Daniel"
user2_id = "John"

In [7]:
def create_user(user_id):
    response = client.create_user(
        CollectionId=collection_id, 
        UserId=user_id,
    )
    print(response)

create_user(user_id)
create_user(user2_id)

AttributeError: 'Rekognition' object has no attribute 'create_user'

### Confirm your collection creation

With the **list_users** method we can see the created users in our collection. 

The **UserStatus** reflects the status of an operation which updates a User representation with a list of given faces. The can be:
- ACTIVE - All associations or disassociations of FaceID(s) for a User are complete.
- CREATED - A User has been created,but has no FaceID(s) associated with it.
- UPDATING - A User is being updated and there are current associations or disassociations of FaceID(s) taking place.

In [9]:
#ListUsers - Lists the users in a collection.
def list_users():
    response = client.list_users(
        CollectionId=collection_id
    )
    print(response["Users"])

list_users()


[{'UserId': 'John', 'UserStatus': 'CREATED'}, {'UserId': 'Daniel', 'UserStatus': 'CREATED'}]


### Add faces to a collection

Now we have our user created, let's populate the face collection with photos which will later be associated to the user. 

In [10]:
def populate_collection(collection, directory):
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            print(f)
            file = open(f, "rb") # opening for [r]eading as [b]inary
            data = file.read() 
            response=client.index_faces(CollectionId=collection,
                                Image={'Bytes':data},
                                ExternalImageId=f.split("/")[2],
                                MaxFaces=1,
                                QualityFilter="AUTO",
                                DetectionAttributes=['ALL'])
            print ('Results for ' + f.split("/")[2])
            print('Faces indexed:')
            for faceRecord in response['FaceRecords']:
                print('  Face ID : {}'.format( faceRecord['Face']['FaceId']))
                print('  Location: {}'.format(faceRecord['Face']['BoundingBox']))

            if len(response['UnindexedFaces']) > 0:
                print('Faces not indexed:')
                for unindexed_face in response['UnindexedFaces']:
                    print(' Location: {}'.format(unindexed_face['FaceDetail']['BoundingBox']))
                    print(' Reasons :')
                    for reason in unindexed_face['Reasons']:
                        print('   ' + reason)
            file.close()
    return

In [11]:
directory = 'media/multiple-faces'

In [12]:
populate_collection(collection_id,directory)

media/multiple-faces/dani1.jpg
Results for dani1.jpg
Faces indexed:
  Face ID : f71eba80-0e18-41a3-8d23-bcf1567443fb
  Location: {'Width': 0.6322183609008789, 'Height': 0.5936641097068787, 'Left': 0.07803937792778015, 'Top': 0.13227343559265137}
media/multiple-faces/dani3.jpg
Results for dani3.jpg
Faces indexed:
  Face ID : bc8747e2-63ad-4871-93f6-08f3bf969e93
  Location: {'Width': 0.6141138672828674, 'Height': 0.5431267023086548, 'Left': 0.12507522106170654, 'Top': 0.10763099789619446}
media/multiple-faces/dani2.jpg
Results for dani2.jpg
Faces indexed:
  Face ID : e502456b-2aca-48c0-b163-32cace0b48c1
  Location: {'Width': 0.5996366739273071, 'Height': 0.580813467502594, 'Left': 0.12779802083969116, 'Top': 0.14054232835769653}
media/multiple-faces/dani4.jpg
Results for dani4.jpg
Faces indexed:
  Face ID : 9d966fa6-b298-4f61-9021-99e08bcbfe41
  Location: {'Width': 0.6733003258705139, 'Height': 0.623356282711029, 'Left': 0.07407209277153015, 'Top': 0.23582562804222107}


### List faces in the collection

Review the faces have been correctly indexed into the collection.

In [13]:
def list_collection_faces(collection_id):
    response = client.list_faces(
        CollectionId=collection_id
    )
    faces = []
    for face in response["Faces"]:
        faces.append(face["FaceId"])
        print("Image: {}, FaceId: {}".format(face["ExternalImageId"],face["FaceId"]))
    return faces

faces = list_collection_faces(collection_id)
print("FaceIds:",faces)

Image: dani4.jpg, FaceId: 9d966fa6-b298-4f61-9021-99e08bcbfe41
Image: dani3.jpg, FaceId: bc8747e2-63ad-4871-93f6-08f3bf969e93
Image: dani2.jpg, FaceId: e502456b-2aca-48c0-b163-32cace0b48c1
Image: dani1.jpg, FaceId: f71eba80-0e18-41a3-8d23-bcf1567443fb
FaceIds: ['9d966fa6-b298-4f61-9021-99e08bcbfe41', 'bc8747e2-63ad-4871-93f6-08f3bf969e93', 'e502456b-2aca-48c0-b163-32cace0b48c1', 'f71eba80-0e18-41a3-8d23-bcf1567443fb']


### Associate faces in the collections to a user

It's time to associate the faces in our collection to our user. For this task we will use the **associate_faces** method.

This method takes an array of FaceIds. Each FaceId that is present in the list is associated with the provided User. The maximum number of total FaceIds per User is 100.

The parameter specifies the minimum User match confidence required for the face to be associated with a User that has at least one faceID already associated. This ensures that the FaceIds are associated with the right User. The value ranges from 0-100 and default value is 75.

#### Associate a single face to a user

Let's associate a single face from our faces array. 

In [14]:
print(faces[0:1])

['9d966fa6-b298-4f61-9021-99e08bcbfe41']


In [15]:
def associate_one_face(faceid, collection_id, user_id):
    response = client.associate_faces(
        CollectionId=collection_id,
        UserId=user_id,
        FaceIds=faceid
    )
    print(response)

associate_one_face(faces[0:1], collection_id, user_id)

{'AssociatedFaces': [{'FaceId': '9d966fa6-b298-4f61-9021-99e08bcbfe41'}], 'UnsuccessfulFaceAssociations': [], 'UserStatus': 'UPDATING', 'ResponseMetadata': {'RequestId': '47ed8825-829e-433e-a678-2762e6d87b2a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '47ed8825-829e-433e-a678-2762e6d87b2a', 'content-type': 'application/x-amz-json-1.1', 'content-length': '129', 'date': 'Wed, 28 Jun 2023 14:11:40 GMT'}, 'RetryAttempts': 0}}


#### Associate multiple faces in the collections to a user

Let's associate a the remaining faces from our faces array

In [16]:
print(faces[1:])

['bc8747e2-63ad-4871-93f6-08f3bf969e93', 'e502456b-2aca-48c0-b163-32cace0b48c1', 'f71eba80-0e18-41a3-8d23-bcf1567443fb']


In [17]:
def associate_multiple_faces(faces, collection_id, user_id):
    response = client.associate_faces(
        CollectionId=collection_id,
        UserId=user_id,
        FaceIds=faces,
        UserMatchThreshold=75
    )
    print(response)

associate_multiple_faces(faces[1:], collection_id, user_id)

{'AssociatedFaces': [{'FaceId': 'bc8747e2-63ad-4871-93f6-08f3bf969e93'}, {'FaceId': 'e502456b-2aca-48c0-b163-32cace0b48c1'}, {'FaceId': 'f71eba80-0e18-41a3-8d23-bcf1567443fb'}], 'UnsuccessfulFaceAssociations': [], 'UserStatus': 'UPDATING', 'ResponseMetadata': {'RequestId': '2deefcca-999b-406a-a12b-94ae568a386b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2deefcca-999b-406a-a12b-94ae568a386b', 'content-type': 'application/x-amz-json-1.1', 'content-length': '229', 'date': 'Wed, 28 Jun 2023 14:12:02 GMT'}, 'RetryAttempts': 0}}


### Search Users by UserId or FaceId

Searches for Users within a collection based on a or UserId. This API can be used to find the closest User (with a highest similarity) to associate a face. 

The request must be provided with either FaceId or UserId. The operation returns an array of User that matches the FaceId or UserId, ordered by similarity score with the highest similarity first.

In [21]:
def search_users(collection_id,face_id):
    response = client.search_users(
        CollectionId=collection_id,
        FaceId=face_id
        #UserId=user_id
    )
    print(response)
    
search_users(collection_id,faces[0:1][0])

{'UserMatches': [{'Similarity': 99.99925231933594, 'User': {'UserId': 'Daniel', 'UserStatus': 'ACTIVE'}}], 'FaceModelVersion': '6', 'SearchedFace': {'FaceId': '9d966fa6-b298-4f61-9021-99e08bcbfe41'}, 'ResponseMetadata': {'RequestId': '61e6891b-9aaf-41ae-ad0e-93af987016ed', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '61e6891b-9aaf-41ae-ad0e-93af987016ed', 'content-type': 'application/x-amz-json-1.1', 'content-length': '187', 'date': 'Wed, 28 Jun 2023 14:15:52 GMT'}, 'RetryAttempts': 0}}


### Search Users by Image

Searches for Users using a supplied image. It first detects the largest face in the image, and then searches a specified collection for matching Users.

The operation returns an array of Users that match the face in the supplied image, ordered by similarity score with the highest similarity first. It also returns a bounding box for the face found in the input image.

In [22]:
image="dani.png"

def search_users_by_image(collection_id,image):
    file = open("media/{}".format(image), "rb") # opening for [r]eading as [b]inary
    data = file.read() 
    response = client.search_users_by_image(
        CollectionId=collection_id,
        Image={'Bytes':data}
    )
    print(response)
    
search_users_by_image(collection_id,image)

{'UserMatches': [{'Similarity': 99.9505615234375, 'User': {'UserId': 'Daniel', 'UserStatus': 'ACTIVE'}}], 'FaceModelVersion': '6', 'SearchedFace': {'FaceDetail': {'BoundingBox': {'Width': 0.47910138964653015, 'Height': 0.457343190908432, 'Left': 0.24557287991046906, 'Top': 0.2843273878097534}}}, 'UnsearchedFaces': [], 'ResponseMetadata': {'RequestId': 'e8faa317-f6c3-46eb-9bb8-4ec4ba976923', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e8faa317-f6c3-46eb-9bb8-4ec4ba976923', 'content-type': 'application/x-amz-json-1.1', 'content-length': '297', 'date': 'Wed, 28 Jun 2023 14:16:20 GMT'}, 'RetryAttempts': 0}}


### Disassociate faces from a user

Remove the association between a Face supplied in an array of FaceIds and the User. If the User is not present already, then a ResourceNotFound exception is thrown.

In [ ]:
def disassociate_faces(face_ids,collection_id,user_id):
    response = client.disassociate_faces(
        CollectionId=collection_id,
        UserId=user_id,
        FaceIds=face_ids
    )
    print(response)
disassociate_faces(faces,collection_id,user_id)

### Delete a user

Let's delete the user we created in our collection.

In [ ]:
def delete_user(collection_id,user_id):
    response = client.delete_user(
        CollectionId=collection_id,
        UserId=user_id
    )
    print(response)

In [ ]:
delete_user(collection_id,user_id)
delete_user(collection_id,user2_id)

In [ ]:
list_users()

### Delete collection

Let's delete the collections we created in our account.

In [ ]:
def delete_collection(collection_id):

    print('Attempting to delete collection ' + collection_id)
    status_code=0
    try:
        response=client.delete_collection(CollectionId=collection_id)
        status_code=response['StatusCode']
        
    except ClientError as e:
        if e.response['Error']['Code'] == 'ResourceNotFoundException':
            print ('The collection ' + collection_id + ' was not found ')
        else:
            print ('Error other than Not Found occurred: ' + e.response['Error']['Message'])
        status_code=e.response['ResponseMetadata']['HTTPStatusCode']
    print('Status code: ' + str(status_code))


delete_collection(collection_id)

In [ ]:
collection_count=list_collections()
print("Collections: " + str(collection_count))